# 문제 상황 요약

- **삼성전자 몰 활성화**를 위해서, 다량의 데이터를 기반으로 한 제품(카테고리) 추천을 해야 함.
- **데이터 규모**: 구매 약 3천만 건, 사용자 데모 정보 3천만 건, 웹 로그 7천만 건 정도로 아주 큼.
- **사용자별 구매 이력 희소성**: 일반적으로 냉장고·TV·스마트폰·스마트워치·오븐 등 3~4개 카테고리밖에 안 삼.
- **추천시 고려**:
  1. 사용자 데모 정보(연령, 성별, 계정 활성도)
  2. 구매/웹로그 시계열(구매 날짜, 웹로그 발생 날짜)
  3. 구매 가격, 구매량

# FM(Factorization Machine) 추천 이유

1. **다양한 피처와 상호작용을 모델링하기 좋음**
   - 유저 정보, 아이템 카테고리, 구매 시점, 구매 가격/수량, 웹로그 특성 등 각각을 피처로 만들어 원-핫이나 임베딩으로 넣으면, FM이 이들의 쌍 상호작용을 자동으로 학습해 준다.
2. **규모가 큰 희소 데이터 처리에 유리**
   - 구매 이력이 적어 한 사용자 기준으론 희소하지만, 전체적으로는 데이터량이 많다. FM은 높은 차원에서 많은 피처를 효과적으로 학습하는 장점을 가진다.
3. **간단한 구조 + 설명 가능**
   - 시퀀스 모델(RNN, Transformer)과 달리 구현이 상대적으로 쉽고, 왜 특정 추천이 나왔는지 해석하기도 편하다. (피처 임베딩 분석 등)

# FM 적용시 장단점

## 장점

- **여러 종류의 피처 통합**: 구매 패턴, 시간정보, 유저 데모 등 다 넣을 수 있음.
- **상호작용 학습**: (유저특성 × 아이템특성), (연령 × 계절), (성별 × 웹로그 방문빈도) 같은 2차항을 모델이 자동으로 파악.
- **희소 데이터 대응**: 원-핫 벡터가 거대해져도, 임베딩으로 수용 가능.
- **설명력**: 시퀀스 모델에 비해, 어떤 피처 쌍이 예측에 크게 기여하는지 해석하기 조금 더 쉽다.

## 단점

- **시계열 순서**를 직접적으로 모델링하기 어려움. "구매 순서" 같은 건 사람이 별도로 피처 엔지니어링을 해야 함.
- **복잡한 상호작용**(3차 이상 교호작용)은 기본 FM으로는 제한적. 더 깊은 패턴을 잡기 위해서는 DeepFM, xDeepFM 같은 확장이 필요.
- **대규모 데이터 학습 파이프라인**: 3천만~7천만 행을 효율적으로 전처리하고 분산/병렬 학습할 환경(예: Spark, Dask 등)이 필수적.

# 결론

- **FM 모델**은 현재 "다양한 피처(데모+시계열+금액+웹로그) 활용"과 "설명력"을 동시에 충족하기에 적합한 선택.
- **시퀀스 모델**에 비해 구현이 수월하고, 대규모 데이터에서도 성능 확보가 가능하다.
- 단, 정말 시계열 순서를 정밀하게 활용하려면 직접 '최근 3개월 구매 여부', '구매 간격' 등 시간 피처를 만들어서 FM에 투입해야 한다.


# Factorization Machine(FM)이란?

**한 마디로**

> 여러 피처(사용자 정보, 제품 카테고리, 가격, 시점 등) 사이의 **상호작용**을 자동으로 학습해,  
> “누가 어떤 아이템을 좋아할 확률이 높은지” 예측해주는 모델!

---

## 1. 왜 FM을 쓰나?

1. **상호작용 자동 학습**
   - 예: 나이=30대 & 카테고리=스마트워치 & 최근 3개월 구매이력=1
   - 각 피처를 일일이 곱해서 모델에 넣기 힘든데, FM은 이걸 임베딩 벡터 내적으로 자동 계산해버림.
2. **희소 데이터에 강함**
   - 사용자 개별 구매 이력은 적지만, 전체적으로 유저·아이템이 많을 때 유리.
3. **실제 업무에 쉽게 붙일 수 있음**
   - 피처를 원-핫 혹은 임베딩 형태로 준비만 잘 하면, FM이 그걸 바로 학습함.

---

## 2. 작동 방식 (간단 버전)

1. 모든 피처(유저, 아이템, 시간, 가격, 성별 등)를 **원-핫** 또는 **수치화(연속형)** 해서 하나의 벡터로 만든다.
2. **임베딩(embedding)**: 각 피처마다 잠재 벡터를 만들어, 피처 사이의 관계를 내적으로 표현.
3. 예측할 때는  
   \[
   \text{(1차 선형합)} + \text{(피처 쌍들의 상호작용)} \rightarrow \text{최종 예측}
   \]

---

## 3. 예시로 생각해보자

- 만약 “30대 남성이 TV를 100만원에 구매”했다면, “(30대 & TV)”, “(남성 & 100만원)”, “(TV & 100만원)” 같은 조합이 중요해질 수 있잖아?
- FM은 이걸 자동으로 (특정 임베딩 벡터들끼리) 내적해서 “이 조합이 얼마나 긍정적인지” 알아낸다.

---

## 4. 우리가 얻을 수 있는 이점

1. **추천 정확도 증가**
   - 단순히 “이 유저가 과거에 본 것”만 보는 게 아니라,  
     나이·성별·가격대·시점 등을 전부 활용해 다각적으로 예측.
2. **확장성**
   - 데이터가 많아도(수천만 건 이상) 분산 학습만 잘 하면 스케일 업 가능.
3. **해석 가능성**
   - “어떤 피처 쌍이 상호작용으로 크게 기여했는지”를 임베딩으로 어느 정도 확인 가능.

---

## 5. 정리

- **Factorization Machine**은 “여러 피처가 결합해 나타나는 효과”를 한 번에 학습해주는 모델.
- 우리처럼 “데이터는 방대하지만, 각 유저 구매 이력은 적고, 데모/시간/가격 등 다양한 피처”를 써야 할 때 딱 맞다.
- 시계열 순서를 복잡하게 분석하긴 어렵지만, “최근 6개월 구매 여부” 등의 피처만 만들어 넣으면 충분히 커버 가능.
- 현업에서 **“협업필터링 + 다양한 피처”**를 함께 쓰고 싶다면 FM 계열이 대표적인 솔루션이다.


# Factorization Machine (FM)이 우리 데이터셋에서 어떻게 작동하나?

## 1. 기본 아이디어

FM은 “특징(피처)들 간의 2차 상호작용”을 효율적으로 학습하는 모델이야.  
우리 문제에서 특징이 되는 것들을 생각해보자:

1. **유저 식별자(아이디)**
2. **유저 데모 정보**: 연령, 성별, 계정 활성도(최근 6개월/1년/비활성) 등
3. **카테고리 정보**: 예) 냉장고, TV, 스마트폰, 워치, 오븐, 식기세척기 등 총 40개
4. **구매 및 웹로그 시점 정보**: 구입/접속 연도, 월, 최근 3개월 구매여부 등
5. **구매 가격, 구매량**: (연속형으로 넣을 수 있음)

FM은 위 피처들을 원-핫 혹은 임베딩 형태로 받아서, “어느 피처와 어느 피처가 같이 나타날 때 예측에 어떤 영향을 미치는가?”를 자동으로 학습한다.

---

## 2. 우리 데이터셋에 적용하는 구조

### 2.1 입력 설계(피처 인코딩)

1. **유저 ID**

   - 3천만 명 수준이라 one-hot으로 하면 차원이 말도 안 되게 커.
   - 그래서 “유저 ID -> 임베딩 index” 방식으로 변환하고, 모델에서는 그 임베딩 벡터를 쓰게 되겠지.
   - FM에서는 임베딩( \(\mathbf{v}_{user}\) )을 만들어서, 아이템(카테고리) 임베딩( \(\mathbf{v}_{category}\) )과의 내적을 통해 ‘유저×카테고리’ 상호작용을 학습한다.

2. **카테고리**

   - 40개 정도라면 one-hot이든 임베딩이든 크게 부담이 없을 수도 있어.
   - “카테고리 = TV” -> (1, 0, 0, ..., 0) 식의 원-핫으로 만들거나, 혹은 40차원 중 하나의 index로 해서 임베딩을 쓸 수도 있음.

3. **데모 정보(연령, 성별, 계정 활성도 등)**

   - 범주형(성별=남/여, 활성도=3개 범주)는 one-hot이 일반적.
   - 연령(예: 10대/20대/30대/…)도 범주로 나눠서 one-hot이든, 혹은 continuous로 분할해서 range별로 처리할 수도 있어.
   - FM에서는 해당 피처들의 임베딩( \(\mathbf{v}_{gender}\), \(\mathbf{v}_{agebucket}\), \(\mathbf{v}\_{accountStatus}\) )을 학습하게 됨.

4. **시계열 정보**

   - “구매 날짜(연, 월, 주), 최근 구매일로부터의 차이, 최근 3개월 내 구매 횟수” 등등을 **새로운 피처**로 만든다.
   - 예) `month_1`, `month_2`, ..., `month_12` 같은 one-hot, 또는 단순히 (0~11) 임베딩을 쓸 수도 있음.
   - “최근 3개월에 구매 이력이 있으면 1, 없으면 0” 식의 이진 피처도 가능.
   - 이렇게 만든 시계열 관련 피처들도 FM이 임베딩 벡터를 가지게 되니, ( “month=1” × “카테고리=에어컨” ) 상호작용 같은 걸 학습할 수 있음.

5. **구매 가격, 구매량**
   - 연속형이라면 그대로 x_i값으로 넣으면 됨. 예: 구매 가격=1,000,000, 구매량=2
   - FM은 각 연속형 피처에도 임베딩 벡터를 할당할 수 있어. 다만 범위를 적절히 스케일링(normalization) 해주는 게 좋을 거야.

---

### 2.2 FM 예측 식의 해석

FM의 예측 식(회귀 혹은 이진 분류를 가정)은 보통 아래 형태야:

$$
\hat{y} = w_0
+ \sum_{i=1}^{n} w_i x_i
+ \frac{1}{2} \sum_{i=1}^{n} \sum_{j=i+1}^{n} \langle \mathbf{v}_i, \mathbf{v}_j \rangle x_i x_j
$$

- 여기서 \(x_i\)는 우리 데이터셋에서의 “유저 ID 임베딩 인덱스, 카테고리, 성별, 나이구간, 구매가격, …” 같은 피처 값.
- \(\mathbf{v}\_i\)는 i번째 피처에 대한 “잠재 벡터(embedding)”.
- \(\langle \mathbf{v}\_i, \mathbf{v}\_j \rangle\)는 벡터 내적. 즉, i와 j 피처의 상호작용 강도.

**예시**: “30대 & 남성 & 최근 6개월 활성 & TV 카테고리 & 구매 가격=100만원”이 동시에 나타났을 때, 어떤 예측값이 나오는지를 FM이 계산한다. 여기서 (30대 × TV) 상호작용, (남성 × 100만원 구매가격) 상호작용 등등이 자동으로 학습되는 거야.

---

## 3. 실제 예측 태스크

실제로는 **유저가 어떤 카테고리를 구매하거나 관심 가질 확률**을 예측하는 식으로 FM을 사용할 거야.

- 모델 입력: (유저, 시간, 카테고리, 데모, 가격/수량, 웹로그 등)
- 모델 출력: 이 샘플(행)에 대해서 **구매(1) vs 비구매(0)** 확률, 혹은 클릭/장바구니 담기 확률 등.

훈련 시에는 “실제로 구매했으면 1, 아니면 0”을 타겟으로 두고 손실 함수(예: Binary Cross Entropy)를 쓴다.

---

## 4. 학습 과정에서 중요한 점

1. **스파스/대규모 데이터 처리**

   - 3천만 유저 × 40 카테고리 × 여러 시간 피처… 피처 공간이 매우 커질 수 있음.
   - 그래서 대부분은 Spark나 Dask, 혹은 분산 PyTorch, 혹은 GPU를 잘 활용하는 기법을 써야 함.

2. **피처 엔지니어링**

   - FM 자체가 시계열 순서를 자동으로 학습하지 않으므로, “최근 구매 시점” 등 시간을 잘 캡슐화한 피처를 만들어 넣어야 한다.
   - 구매량, 웹로그 방문 횟수 같은 수치를 log 스케일이나 min-max 스케일링으로 적절히 변환하는 것도 성능과 안정성에 도움.

3. **하이퍼파라미터**
   - 임베딩 차원 k(예: 8, 16 등)를 어떻게 정하느냐에 따라 모델 복잡도가 달라짐.
   - 학습률, 정규화(regularization) 파라미터도 튜닝 대상.

---

## 5. 예측 및 활용 방법

1. **카테고리 추천**

   - “한 명의 유저”에 대해 40개 카테고리에 대한 예측값(구매 확률 혹은 스코어)을 FM으로 전부 뽑은 뒤, 그중 상위 N개를 추천.
   - 예: 이 유저는 TV보다는 식기세척기 확률이 높게 나오면 식기세척기를 우선 추천.

2. **캠페인 대상 선정**

   - 광고나 프로모션을 돌릴 때, FM 스코어가 일정 이상인 유저를 타겟팅할 수 있음.

3. **설명/해석**
   - FM 모델은 \(\mathbf{v}\_i\)라는 임베딩 벡터를 학습하므로, “어떤 피처(예: 30대)와 어떤 피처(예: 오븐) 간 내적이 높은지”를 보면, 그 조합이 유의미한 상호작용임을 알 수 있다.
   - \(\langle \mathbf{v}_{성별=남}, \mathbf{v}_{스마트폰} \rangle\)가 매우 크다면, 남성이 스마트폰을 살 확률이 더 높다는 식으로 설명 가능.

---

## 6. 요약

1. **우리 데이터셋 특성**

   - 유저별 구매 이력이 적지만, 전체적으로는 피처가 다양하고 로그 규모가 크다.
   - 데모 정보, 구매/웹로그 시계열, 가격/수량 정보를 모두 잘 반영해야 함.

2. **FM이 작동하는 방식**

   - 피처(유저, 카테고리, 시간, 데모, 가격 등)를 임베딩화하여, 이들의 2차 상호작용(내적)을 통해 예측값을 산출한다.
   - 원-핫된 대규모 sparse 피처에도 효과적으로 대응한다.

3. **실제 프로세스**

   - (1) 데이터 전처리: 피처 인코딩(원-핫, 임베딩 인덱스, 연속형 변환 등)
   - (2) FM 학습: Binary Classification(구매=1, 비구매=0)로 학습 (또는 회귀로 점수를 예측)
   - (3) 추천 점수 산출 & 상위 N개 카테고리 추천.

4. **주의점**
   - 시계열의 순서를 자동으로 잡아주진 않기에, “최근 구매 시점” 같은 변수를 만들어야 함.
   - 하드웨어 인프라와 분산 환경에서의 학습 파이프라인이 중요(데이터가 너무 크니까).

결국 **FM**은 “희소 데이터 + 다양한 피처 상호작용” 상황에 매우 잘 맞고, 시계열 변수도 잘 엔지니어링해서 넣으면 큰 성과가 있을 거야.


In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# -------------------------
# 1. 샘플 데이터 생성
# -------------------------
def generate_sample_data(num_samples=10000):
    """
    간단한 synthetic 데이터 생성:
      - user_id: 0~99 (100명)
      - category_id: 0~9 (10개 카테고리)
      - gender: 0/1
      - age_bucket: 0~4 (20대/30대/40대/50대/60대+ 같은 느낌)
      - price: 구매가격 (연속값)
      - label: 구매 여부(0/1)
    """
    np.random.seed(42)
    
    user_ids = np.random.randint(0, 100, size=num_samples)
    category_ids = np.random.randint(0, 10, size=num_samples)
    gender = np.random.randint(0, 2, size=num_samples)
    age_bucket = np.random.randint(0, 5, size=num_samples)
    price = np.random.randint(1, 1000, size=num_samples).astype(np.float32)
    
    # 간단히 몇 가지 규칙을 부여해 label을 생성해보자
    # 예: (카테고리==3 이면서 price>500)면 구매확률이 높게 하는 식
    # 실제론 훨씬 복잡하지만, 여기선 "데모" 목적
    prob = 0.1 * np.ones(num_samples)
    prob += 0.2 * (category_ids == 3)  # 특정 카테고리(3)면 가중
    prob += 0.2 * (gender == 1)       # gender=1인 경우 약간 가중
    prob += 0.2 * (price > 500)       # 가격이 500 이상이면 가중
    # 확률을 [0,1] 범위로 클리핑
    prob = np.clip(prob, 0, 1)
    
    labels = np.random.binomial(1, prob, size=num_samples)
    
    # 이제 feature를 one-hot (또는 임베딩 인덱스)로 만들 수도 있지만
    # 여기선 간단히 "원-핫으로 펼쳐서" 사용해보자
    # user_id: 100차원 one-hot
    user_onehot = np.zeros((num_samples, 100))
    user_onehot[np.arange(num_samples), user_ids] = 1
    
    # category_id: 10차원 one-hot
    cat_onehot = np.zeros((num_samples, 10))
    cat_onehot[np.arange(num_samples), category_ids] = 1
    
    # gender: 2차원 one-hot
    gen_onehot = np.zeros((num_samples, 2))
    gen_onehot[np.arange(num_samples), gender] = 1
    
    # age_bucket: 5차원 one-hot
    age_onehot = np.zeros((num_samples, 5))
    age_onehot[np.arange(num_samples), age_bucket] = 1
    
    # price: 연속형 -> 여기선 그냥 한 칼럼으로 둠
    # (스케일링 하는 것도 좋지만, 데모니까 패스)
    price_reshaped = price.reshape(-1, 1)
    
    # 최종 X는 모든 걸 이어붙인 벡터
    X = np.hstack([
        user_onehot,
        cat_onehot,
        gen_onehot,
        age_onehot,
        price_reshaped
    ])
    
    return X, labels

# -------------------------
# 2. FM Layer (직접 구현)
# -------------------------
class FactorizationMachineLayer(layers.Layer):
    """
    (batch_size, n_features) 입력에 대해
    FM의 1차 + 2차 상호작용을 계산하는 Layer.
    """
    def __init__(self, n_features, k=8, **kwargs):
        super().__init__(**kwargs)
        self.n_features = n_features
        self.k = k
    
    def build(self, input_shape):
        # 1차 항 (linear term) 파라미터
        self.w = self.add_weight(
            name='linear_w',
            shape=(self.n_features, 1),
            initializer='glorot_uniform',
            trainable=True
        )
        # 편향
        self.b = self.add_weight(
            name='bias',
            shape=(1,),
            initializer='zeros',
            trainable=True
        )
        # 임베딩(2차 상호작용 용)
        self.V = self.add_weight(
            name='embedding_V',
            shape=(self.n_features, self.k),
            initializer='glorot_uniform',
            trainable=True
        )
        super().build(input_shape)

    def call(self, inputs):
        """
        inputs: (batch_size, n_features)
        """
        # 1) 1차항
        linear_part = tf.matmul(inputs, self.w) + self.b  # (batch_size, 1)
        
        # 2) 2차 상호작용
        # (V * x)의 제곱 - (V^2 * x^2)
        # inputs.shape = (batch_size, n_features)
        # V.shape = (n_features, k)
        
        # (batch_size, k)
        XV = tf.matmul(inputs, self.V)
        XV_square = tf.square(XV)  # (batch_size, k)
        
        # (batch_size, n_features)
        X_square = tf.square(inputs)
        V_square = tf.square(self.V)  # (n_features, k)
        
        # (batch_size, k)
        X_V_square = tf.matmul(X_square, V_square)
        
        # (batch_size, )
        interaction_part = 0.5 * tf.reduce_sum(XV_square - X_V_square, axis=1, keepdims=True)
        
        # 예측값 = 1차 + 2차
        outputs = linear_part + interaction_part  # (batch_size, 1)
        return outputs

# -------------------------
# 3. FM Model 구성
# -------------------------
def create_fm_model(n_features, k=8):
    inputs = keras.Input(shape=(n_features,))
    fm_output = FactorizationMachineLayer(n_features=n_features, k=k)(inputs)
    # 여기서는 로짓(logit) 형태로 나오니, 시그모이드를 통과시키면 확률 예측
    outputs = layers.Activation('sigmoid')(fm_output)
    
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

# -------------------------
# 4. 학습 시연
# -------------------------
if __name__ == "__main__":
    # 4.1 데이터 준비
    X, y = generate_sample_data(num_samples=10000)
    print("X shape:", X.shape, "y shape:", y.shape)
    # (10000, 100 + 10 + 2 + 5 + 1) = (10000, 118)

    # 4.2 모델 생성
    n_features = X.shape[1]
    fm_model = create_fm_model(n_features, k=8)
    fm_model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=0.01),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    fm_model.summary()

    # 4.3 학습
    history = fm_model.fit(
        X, y,
        validation_split=0.2,
        batch_size=128,
        epochs=5,
        verbose=1
    )

    # 4.4 간단 예측
    # 임의로 5개만 뽑아서 확률 예측
    preds = fm_model.predict(X[:5])
    print("Sample preds:", preds.flatten())
    print("Real labels: ", y[:5])


X shape: (10000, 118) y shape: (10000,)


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 118)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ factorization_machine_layer     │ (None, 1)              │         1,063 │
│ (FactorizationMachineLayer)     │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 1)              │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,063 (4.15 KB)

 Trainable params: 1,063 (4.15 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5589 - loss: 12.5998 - val_accuracy: 0.6720 - val_loss: 0.7905
Epoch 2/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 493us/step - accuracy: 0.6419 - loss: 0.7888 - val_accuracy: 0.6115 - val_loss: 0.7830
Epoch 3/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 472us/step - accuracy: 0.6453 - loss: 0.7527 - val_accuracy: 0.5655 - val_loss: 1.0233
Epoch 4/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 464us/step - accuracy: 0.6538 - loss: 0.7569 - val_accuracy: 0.6585 - val_loss: 0.6475
Epoch 5/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 466us/step - accuracy: 0.6577 - loss: 0.6789 - val_accuracy: 0.5785 - val_loss: 0.8653
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Sample preds: [0.27413014 0.5641592  0.76458025 0.89758974 0.51780593]
Real labels:  [0 0 1 0 1]


# Factorization Machine(FM) 수식 정리

## 1. FM의 전체 예측식

FM(Factorization Machine)은 입력 피처 벡터 \(\mathbf{x} = (x_1, x_2, \dots, x_n)\)에 대해  
아래와 같은 형태로 예측값 \(\hat{y}(\mathbf{x})\)를 계산한다:

$$
\hat{y}(\mathbf{x})
= w_0
+ \sum_{i=1}^{n} w_i x_i
+ \frac{1}{2}\sum_{i=1}^{n} \sum_{j=i+1}^{n} \langle \mathbf{v}_i, \mathbf{v}_j \rangle \; x_i x_j.
$$

- \(w_0\): 전체 편향(bias)
- \(w_i\): 1차 가중치(linear term)
- \(\mathbf{v}\_i \in \mathbb{R}^k\): \(i\)번째 피처의 임베딩(잠재 벡터)
- \(\langle \mathbf{v}\_i, \mathbf{v}\_j \rangle\): 벡터 \(\mathbf{v}\_i\)와 \(\mathbf{v}\_j\)의 내적(dot product)

내적은 보통 다음과 같이 쓴다:

$$
\langle \mathbf{v}_i, \mathbf{v}_j \rangle
= \sum_{f=1}^{k} v_{i,f} \cdot v_{j,f}.
$$

---

## 2. 2차 상호작용 항의 효율적 계산

보통 \(\mathcal{O}(n^2)\)번 곱셈이 필요한 2차 항을, 아래와 같이 변형해서 \(\mathcal{O}(nk)\)로 계산할 수 있다:

$$
\frac{1}{2}\sum_{i=1}^{n}\sum_{j=i+1}^{n} \langle \mathbf{v}_i, \mathbf{v}_j \rangle x_i x_j
=
\frac{1}{2}
\Bigg[
\sum_{f=1}^{k}
\Big(
\sum_{i=1}^{n} v_{i,f} x_i
\Big)^2
\;-\;
\sum_{i=1}^{n}\sum_{f=1}^{k} v_{i,f}^2\, x_i^2
\Bigg].
$$

여기서 \(v\_{i,f}\)는 \(i\)번째 피처 임베딩 벡터에서 \(f\)번째 차원에 해당하는 값이다.

---

## 3. 유저아이디 임베딩 개념 (예시)

범주형(카테고리) 피처가 아주 많을 때, 원-핫으로 사용하면 차원이 매우 커진다.  
이를 해결하기 위해, 다음처럼 유저아이디(범주)를 임베딩 벡터로 매핑해준다:

$$
\mathbf{u}_{\text{user}} = \mathbf{V}_{\text{user}}(\text{UserID}),
\quad
\mathbf{v}_{\text{item}} = \mathbf{V}_{\text{item}}(\text{ItemID}).
$$

이 임베딩들은 학습하면서 업데이트되고, FM의 상호작용 항에서  
\(\mathbf{u}_{\text{user}} \cdot \mathbf{v}_{\text{item}}\) (내적) 같은 값을 통해  
“유저 × 아이템” 조합이 얼마나 중요한지 자동으로 배우게 된다.

---

## 4. 요약

결국 FM의 예측값은

$$
\hat{y}(\mathbf{x})
= w_0
+ \sum_{i=1}^{n} w_i x_i
+ \frac{1}{2}\sum_{i=1}^{n} \sum_{j=i+1}^{n} \langle \mathbf{v}_i, \mathbf{v}_j \rangle \; x_i x_j.
$$

- **\(w_0\)**: 전역 편향 (베이스라인)
- **\(\sum w_i x_i\)**: 각 피처의 단일 효과
- **\(\langle \mathbf{v}\_i, \mathbf{v}\_j \rangle x_i x_j\)**: 피처 쌍 간 상호작용
- **\(\mathbf{v}\_i\) (임베딩 벡터)**: 유저/아이템 등 범주형 피처의 잠재 벡터.  
  이로써 희소하고 대규모 범주를 효과적으로 표현하고, 다차원 상호작용을 학습한다.


| 모델                           | 장점                                                                                                                                                                                                                                                               | 단점                                                                                                                                                                                                                                                 |
| ------------------------------ | ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------ | ---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **Factorization Machine (FM)** | - **피처 상호작용**(2차 항)을 자동으로 잘 잡아서, 희소 데이터에도 좋음 <br/> - **데모·시계열·가격** 등 다양한 피처를 원핫·임베딩으로 넣기 쉽고, 설명력도 비교적 나쁘지 않음 <br/> - **구현 난이도**가 Transformer보다는 낮은 편                                    | - **시계열 순서** 자체를 자동 학습하긴 어렵고, 직접 "최근 3개월 구매" 같은 변수를 만들어줘야 함 <br/> - 3차 이상 복잡한 상호작용을 기본 FM 만으로는 충분히 못 잡음(DeepFM 등 추가 확장 필요)                                                         |
| **XGBoost**                    | - **의사결정나무** 기반 부스팅이라, 결측값·범주형 데이터를 어느 정도 잘 핸들링 <br/> - 피처 중요도, 의사결정 경로 등으로 **설명력**이 괜찮은 편 <br/> - **탐색적 분석**이나 상대적으로 소규모~중간 규모 데이터에서 빠르게 성능 확보 가능                           | - 범주형 데이터가 매우 클 때 원핫으로 처리하면 차원이 커짐, 속도 저하 <br/> - 시계열 패턴을 자동으로 포착하긴 제한적이라, 타임 피처 엔지니어링 필요 <br/> - 매우 대규모 데이터(억 단위)에서 학습 속도가 Transformer나 FM 특화 모델 대비 느릴 수 있음 |
| **Transformer**                | - **시퀀스(순서) 정보**를 Attention 메커니즘으로 잘 포착, 시계열·텍스트 등 폭넓은 영역에서 성능 탁월 <br/> - Self-Attention으로 멀리 떨어진 피처나 이벤트 간의 상호작용도 자동 학습 <br/> - **병렬화**가 RNN류보다 쉬워서, 규모가 큰 시퀀스 데이터에도 확장성 높음 | - **구현 복잡도** 높고, 튜닝 요소가 많음(레이어 수, 헤드 수, 마스킹 등) <br/> - 중간중간 Attention 계산으로 인해 메모리 사용량이 상당함 <br/> - 완전히 희소한 범주형 데이터를 다룰 때는 별도 임베딩·입력 전처리가 필요해, FM만큼 간단하진 않음       |
